# Shifting AusEFlux

Move results into `ub8` and prepare them for publishing through OpenDAP

Entire the variable name and the version below and the code will do the rest

In [ ]:
import os
import xarray as xr
import numpy as np
from odc.geo.xr import assign_crs

## Parameters

In [ ]:
var='ET'
version='v1.1'

## Open data

In [ ]:
if version=='v1.2':# changing versioning to "v2"
    export=f'/g/data/ub8/au/AusEFlux/v2/{var}/'
else:
    export=f'/g/data/ub8/au/AusEFlux/v1/{var}/'

if version == 'v1.1':
    path = f'/g/data/os22/chad_tmp/NEE_modelling/results/predictions/AusEFlux_{var}_2003_2022_1km_quantiles_v1.1.nc'
    ds = xr.open_dataset(path)
    ds = assign_crs(ds, crs='EPSG:4326')
    
if version == 'v1.2':
    folder = f'/g/data/xc0/project/AusEFlux/results/AusEFlux/{var}/'
    files = [f'{folder}/{i}' for i in os.listdir(folder) if i.endswith(".nc")]
    files.sort()
    
    #combine annual files into one file
    ds = xr.open_mfdataset(files).sel(time=slice('2003','2023'))
    ds = assign_crs(ds, crs='EPSG:4326')
ds

In [ ]:
# ds[f'{var}_median'].isel(time=-1).plot.imshow(robust=True)

## Set up netcdf attributes

In [ ]:
fluxes = {
    'GPP':'Gross Primary Productivity',
    'NEE':'Net Ecosystem Exchange',
    'ER':'Ecosystem Respiration',
    'ET':'Evapotranspiration'
         }

if var =='ET':
    units = 'mm/month'
else:
    units = 'gC/m\N{SUPERSCRIPT TWO}/month'
    
description = f'AusEFlux {var} is created by empirically upscaling the OzFlux eddy covariance network using machine learning methods coupled with climate and remote sensing datasets. The estimates provided within this dataset were extracted from an ensemble of predictions and represent the median and uncertainty range.'

# Create attributes dictionary
attrs_dict={}
attrs_dict['nodata'] = np.nan
attrs_dict['crs'] = 'EPSG:4326'
attrs_dict['short_name'] = var
attrs_dict['long_name'] = fluxes[var]
attrs_dict['units'] = units
if version=='v1.1':
    attrs_dict['version'] = 'v1.1'
else:
    attrs_dict['version'] = 'v2.0'
attrs_dict['description'] = description

ds.attrs = attrs_dict
ds

## Export annual files

and adjust the time dimension to work with OpenDAP (doesn't like int64)

In [ ]:
for v in ds.data_vars:
    del ds[v].attrs['grid_mapping'] #remove this attr

In [ ]:
#list of years and export
years = [str(i) for i in range(2003, 2022+1)]
for year in years:
    xx = ds.sel(time=year)
    
    # hack to make time dim work with OpenDAP which doesn't like datetime64
    start_time = xx.time.values[0].astype('datetime64[D]')###first date
    print(start_time)
    # set time as the duration between actual and first date
    coords_time = np.array(xx.time, dtype='datetime64[D]') - np.array(xx.time, dtype='datetime64[D]')[0]        
    xx['time'] = coords_time.astype('int32')
    xx.time.attrs = {'units': f'days since {start_time}'} #make sure attrs explain int32 time

    if version=='v1.2':
        xx.to_netcdf(f'{export}/AusEFlux_{var}_1km_quantiles_{year}_v2.0.nc')
    else:
        xx.to_netcdf(f'{export}/AusEFlux_{var}_1km_quantiles_{year}_v1.1.nc')